In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer
ce_rerank = SentenceTransformer('sberbank-ai/sbert_large_nlu_ru')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
class Gamer:

    def __init__(self, name, login, gender, age, description, wishes):

        self.name = name
        self.login = login
        self.gender = gender
        self.age = age
        self.description = description
        self.wishes = wishes
        self.get_descr_encoding()

    def get_descr_encoding(self):
        self.descr_encoding = ce_rerank.encode(self.description)

In [ ]:
def parse_login(login):
    url = f'https://lfg.su/user/{login}'
    page = requests.get(url)
    page_soup = BeautifulSoup(page.text, 'html.parser')

    friends_tag = page_soup.find('div', {'class': 'ow_lp_avatars'})
    friends = [f['href'].split('/')[-1] for f in friends_tag.find_all('a')]

    info  = page_soup.find('div', {'class': 'user_profile_data'}).find_all('table')
#     print(info)
    info_dict = {}
    for table in info:
        table_info = table.find_all('tr')
        for tr in table_info:
            if len(tr.find_all('td')) == 2:
                i = tr.find_all('td')
                info_dict[i[0].text] = i[1].text.replace("\n", "")

    return friends, info_dict

def main(users_logins, depth=0):
    global gamers, targets
#     print(users_logins)

    ignore_info = [
        'Дата активности',
        'Возраст',
        'Имя',
        'Я',
        'Кого ищу',
        'День рождения моего избранника',
        'Цель поиска'
    ]
#     url = 'https://lfg.su/users/search-result/latest_activity'
#     main_page = requests.get(url)
#     main_page_soup = BeautifulSoup(main_page.text, 'html.parser')

#     users = main_page_soup.find_all('div', {'class': 'ow_photo_item_wrap'})
#     print(users)
#     users_logins = [user.find('a', {'class': 'ow_photo_userlist_info'})['href'].split('/')[-1] for user in users]

    for login in users_logins:
        print(' '*depth + login)
        friends, info = parse_login(login)
#         print(login, friends)
        age = int(info['Возраст'].split()[0])
        name = info['Имя']
        gender = info['Я']
        wishes = {
            'gender': info['Кого ищу'],
            'age_low': int(info['День рождения моего избранника'].split()[1]),
            'age_high': int(info['День рождения моего избранника'].split()[3]),
            'purpose': info['Цель поиска'].split(', ')
        }
        description = '\n'.join([f'{k}: {v}' for k, v in info.items() if k not in ignore_info])
        if description.strip().replace(' ', '') != '':
            gamers[login] = Gamer(name=name, login=login, gender=gender, age=age, description=description, wishes=wishes)

        if depth <= 100:
            for friend_login in friends:
                targets.append((login, friend_login))
                if friend_login not in gamers.keys():
                    main([friend_login], depth=depth+1)
    return users_logins

gamers = {}
targets = []

main(['goodman_dd','LizMajers', 'google_user_3856', 'Razvedosa', 'Meowmeow', 'HanteRina',
      'Meinumo', 'google_user_2860', 'gawric','GenaVasiliev','Akigis',
      'Pranik','Electronik','Cogneter','c4'
     ])

goodman_dd
 Literally
  Cogneter
   KanZaki
   Sh0ckeR44
    google_user_2853
   Anastasia123
   google_user_2854
LizMajers
 Rexolik
  Your_Sister
 google_user_3319
  NastjaIvanova
   google_user_2732
    Sapphire362
     Kreyvus
      AnnaVajt
       google_user_2386
       hayaati77
        CHYHKA
         EgorHabarov
         XXX69
         Asker
        Darkness
       fr0st2o11
       DenisFeofanov
       theFUM3
     Tk095
     HSTT
     VladHandler
     Kohas
      Denis12345667
       YonakaHitsugaya
     google_user_2716
      Kaita
       google_user_3429
        Meowmeow
         JackLipat
          google_user_3616
           JUnusFajzaliev
           Pranik
            google_user_2860
           google_user_3759
         NikitaDunaevskij
          RainbowFox
           GenaVasiliev
            Razvedosa
             RomanSaraev
              HanteRina
             Dima27_
             gawric
              google_user_4365
               google_user_3856
              goog

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
len(gamers)

128

In [ ]:
cols = ['name', 'login', 'gender','age','description','wishes']
df = pd.DataFrame(columns=cols)
for i in list(gamers.keys()):
    lst = []
    name = gamers[i].name
    login = gamers[i].login
    gender = gamers[i].gender
    age = gamers[i].age
    description = gamers[i].description
    wishes = gamers[i].wishes
    lst.append([name,login,gender,age,description,wishes])
    df = pd.concat([pd.DataFrame(lst, columns=df.columns), df], ignore_index=True)

    print(i)


df

goodman_dd
Literally
Cogneter
KanZaki
Sh0ckeR44
google_user_2853
google_user_2854
LizMajers
Rexolik
Your_Sister
google_user_3319
NastjaIvanova
google_user_2732
Sapphire362
Kreyvus
AnnaVajt
google_user_2386
hayaati77
CHYHKA
EgorHabarov
XXX69
Asker
Darkness
fr0st2o11
DenisFeofanov
theFUM3
Tk095
HSTT
VladHandler
Kohas
Denis12345667
YonakaHitsugaya
google_user_2716
Kaita
google_user_3429
Meowmeow
JackLipat
google_user_3616
JUnusFajzaliev
Pranik
google_user_2860
google_user_3759
NikitaDunaevskij
RainbowFox
GenaVasiliev
Razvedosa
RomanSaraev
HanteRina
Dima27_
gawric
google_user_4365
google_user_3856
google_user_4487
DenisMalyshev
WatermelonSeal
MrOkulus
google_user_4673
IljaCHivkin
Simmetric
google_user_4692
Enclave
DemoNova
OlegSkorobogatov
_Devil_
Combobreaker
MadGamer
Shax
Granix
google_user_1768
Yuji
rb201513
google_user_3583
empty
Kotoponchik
UED
Alica_Melica
EHduardObolenskij
VladislavMartin
KirillLuckevich
DenisJUrnero
DanilAstapenko
AjdosGulev
Gfg
hewie
KsenijaLebedeva
Halcon
RobotUn

,name,login,gender,age,description,wishes
0,Илья,c4,Парень,37,Обо мне: Сижу дома и ничего не делаю. )\nМои и...,"{'gender': 'Девушка', 'age_low': 18, 'age_high..."
1,Ronan Djohua (Вадим),RoDjo,Парень,27,Обо мне: Тихо и мирно предпринимаю попытки (пр...,"{'gender': 'Парень, Девушка', 'age_low': 16, '..."
2,Krator1,Krator1,Парень,26,"Обо мне: Обычный парень, чуть-чуть хиккан,боль...","{'gender': 'Парень, Девушка', 'age_low': 16, '..."
3,Baka_Sobacka,Baka_Sobacka,Парень,22,Обо мне: Одинокий романтик. В комнате полной л...,"{'gender': 'Парень', 'age_low': 20, 'age_high'..."
4,Electronik,Electronik,Парень,31,В чём я хорош: Не требователен к навыку/игрово...,"{'gender': 'Парень, Девушка', 'age_low': 20, '..."
...,...,...,...,...,...,...
123,Sh0ckeR44,Sh0ckeR44,Парень,37,"Обо мне: Геймер, ретро геймер, стример, блогер...","{'gender': 'Парень, Девушка', 'age_low': 17, '..."
124,KanZaki,KanZaki,Девушка,27,"Город: Тверь, Тверская обл., Россия\nПлатформы...","{'gender': 'Парень, Девушка', 'age_low': 20, '..."
125,Александр,Cogneter,Парень,33,"Обо мне: Создал этот сайт, пытаюсь его развива...","{'gender': 'Девушка', 'age_low': 18, 'age_high..."
126,Literally,Literally,Девушка,26,"Город: Москва, Россия\nЛюбимые игры: League of...","{'gender': 'Парень', 'age_low': 20, 'age_high'..."


In [ ]:
df.to_csv('gamers.csv', index=False)

In [ ]:
import pandas as pd
import itertools
from tqdm import tqdm

l = gamers[list(gamers.keys())[0]].descr_encoding.shape[0]

data = pd.DataFrame(columns=[f'gamer_1_feature_{i}' for i in range(l)]+[f'gamer_2_feature_{i}' for i in range(l)] + ['target'])
data_list = []
for target in tqdm(list(itertools.permutations(list(gamers.keys()), 2))):
    data_list.append([*gamers[target[0]].descr_encoding.tolist(), *gamers[target[1]].descr_encoding.tolist(), int(target in targets)])
data = pd.DataFrame(data=data_list, columns=[f'gamer_1_feature_{i}' for i in range(l)]+[f'gamer_2_feature_{i}' for i in range(l)] + ['target'])

100%|██████████████████████████████████| 16256/16256 [00:00<00:00, 18739.77it/s]


In [ ]:
data

,gamer_1_feature_0,gamer_1_feature_1,gamer_1_feature_2,gamer_1_feature_3,gamer_1_feature_4,gamer_1_feature_5,gamer_1_feature_6,gamer_1_feature_7,gamer_1_feature_8,gamer_1_feature_9,...,gamer_2_feature_1015,gamer_2_feature_1016,gamer_2_feature_1017,gamer_2_feature_1018,gamer_2_feature_1019,gamer_2_feature_1020,gamer_2_feature_1021,gamer_2_feature_1022,gamer_2_feature_1023,target
0,0.762278,-0.210248,-0.653229,-0.370006,-0.468354,-0.262080,-0.830133,0.132753,0.113501,0.213539,...,0.454407,0.305245,0.344710,-0.768898,0.068364,-0.002311,-0.293559,0.597492,0.366705,1
1,0.762278,-0.210248,-0.653229,-0.370006,-0.468354,-0.262080,-0.830133,0.132753,0.113501,0.213539,...,0.431263,0.531618,0.589068,0.122646,0.320491,0.120290,-0.097033,0.478840,-0.314241,0
2,0.762278,-0.210248,-0.653229,-0.370006,-0.468354,-0.262080,-0.830133,0.132753,0.113501,0.213539,...,0.741500,0.030348,0.050144,-0.385078,0.226546,-0.394641,0.029884,1.033026,0.054134,0
3,0.762278,-0.210248,-0.653229,-0.370006,-0.468354,-0.262080,-0.830133,0.132753,0.113501,0.213539,...,0.510779,0.329061,1.343214,-0.054286,0.454075,-0.306244,0.098348,0.087895,-0.222555,0
4,0.762278,-0.210248,-0.653229,-0.370006,-0.468354,-0.262080,-0.830133,0.132753,0.113501,0.213539,...,0.778786,0.687945,-0.108270,0.455059,0.318703,-0.125894,-0.238873,0.001441,0.431343,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16251,0.904159,-0.153234,-0.597118,-1.074872,-0.604871,-0.247912,-0.708500,-0.336954,0.079247,-0.114361,...,0.709387,0.374613,-0.320428,-1.029466,0.190245,0.110623,-0.127191,0.491125,0.371636,0
16252,0.904159,-0.153234,-0.597118,-1.074872,-0.604871,-0.247912,-0.708500,-0.336954,0.079247,-0.114361,...,0.769926,0.449610,-0.000521,-0.467158,0.209772,-0.592418,0.021501,0.153188,-0.101770,0
16253,0.904159,-0.153234,-0.597118,-1.074872,-0.604871,-0.247912,-0.708500,-0.336954,0.079247,-0.114361,...,0.751706,0.257069,0.752276,-0.288665,0.115528,0.062986,0.339114,0.269720,-0.232781,0
16254,0.904159,-0.153234,-0.597118,-1.074872,-0.604871,-0.247912,-0.708500,-0.336954,0.079247,-0.114361,...,0.688439,0.769108,-0.076815,-0.120848,-0.008032,-0.235928,-0.001590,0.215632,-0.428650,0


In [ ]:
data.to_csv('gamers_encoded.csv', index=False)

In [ ]:
data['target'].value_counts()

0    15986
1      270
Name: target, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X, y = data.drop(columns=['target']), data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class NNRanker(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fl1 = nn.Linear(input_size, hidden_size)
        self.act1 = nn.Tanh()
        self.fl3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fl1(x)
        x = self.act1(x)
        x = self.fl3(x)
        return x

In [ ]:
class DatasetTeamder(Dataset):

    def __init__(self, X, y, embedding_size):
        self.embedding_size = embedding_size
        self.X = X
        self.Y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        x = self.X.iloc[index]
        y = self.Y.iloc[index]
        return torch.FloatTensor(x), torch.FloatTensor([y])

In [ ]:
train_dataset = DatasetTeamder(X_train, y_train, embedding_size=1024)
validate_dataset = DatasetTeamder(X_val, y_val, embedding_size=1024)
test_dataset = DatasetTeamder(X_test, y_test, embedding_size=1024)

In [ ]:
batch_size=64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validate_loader = DataLoader(validate_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
device = 'cuda'

In [ ]:
input_size = 1024 * 2
hidden_size = 512
output_size = 2

pos_weight = torch.tensor([1, 50]).float()

model = NNRanker(input_size=input_size, hidden_size=hidden_size, output_size=output_size)
model.to(device)
criterion = nn.CrossEntropyLoss(weight=pos_weight)
optimiser = torch.optim.Adam(params=model.parameters(), lr=0.0000001)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
from tqdm import tqdm
epochs = 20

for epoch in range(epochs):
    model.train()

    epoch_loss = []
    preds, targets = [], []
    for x, target in tqdm(train_loader, desc=f'Epoch-{epoch}'):
        optimiser.zero_grad()
        output = model.forward(x.to(device))
#         print(output)
#         print(target)
        loss = criterion(output.cpu(), target.flatten().long())
        epoch_loss.append(loss.item())
        preds.append(output.cpu().argmax(dim=1))
        targets.append(target.flatten().long())
        optimiser.step()

    model.eval()
    epoch_loss_val = []
    for x, target in validate_loader:
        output = model.forward(x.to(device))
        loss = criterion(output.cpu(), target.flatten().long())
        epoch_loss_val.append(loss.item())


    print(f'Epoch: {epoch} | Train loss {sum(epoch_loss)/len(epoch_loss):.5f} | Val loss {sum(epoch_loss_val)/len(epoch_loss_val):.5f} | Train acc {(torch.concat(preds) == torch.concat(targets)).sum()/len(torch.concat(targets))}')


Epoch-0:   0%|                                          | 0/143 [00:00<?, ?it/s]


NameError: name 'optimiser' is not defined

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

gamers_keys = [nick for nick in gamers]

svd = TruncatedSVD(n_components=1024)
svd_matrix = svd.fit_transform([gamers[nick].descr_encoding for nick in gamers_keys])

# Calculate the cosine similarity
# between all pairs of users
cosine_sim = cosine_similarity(svd_matrix)

In [ ]:
a = svd.transform([gamers['google_user_3616'].descr_encoding])

In [ ]:
g = parse_one_person('KirillLuckevich')
b = svd.transform([g.descr_encoding])

In [ ]:
print(g.description)

Обо мне: Я ЗДОРОВЕННЫЙ болтливый и вежливый. Я устал убивать драконов и сбегать из царства мертвых. Просто хочу найти девушку и поболтать, тысячу лет уже не разговаривал с людьми, а с живыми девушками - тем более. А там как пойдет. Да и всю любовь мира не вместить в одного кота, правда ведь?
Мои интересы: Видеоигры, куда же без них. Сражался за Аиур, шлепал Азари и так далее до бесконечности. Люблю трехмерную графику, но трачу на нее недостаточно времени.Фентези и фантастика Всех видов форм и расцветок.Словесные ролевые.МУЗЫКА.
В чём я хорош: Болтать, все кого я знаю утверждают что язык у меня подвешен как надо. Да и в любой непонятной ситуации я привык отшучиваться.Быт, стирки-уборки-готовки.Фантазирование и придумывание всякого, даже трезвый.
Город: Санкт-Петербург, Россия
Любимые жанры игр: Adventure, Quest, RPG, Strategy, MOBA
Предпочтительная сложность игр: Средняя
Платформы: Компьютер, Смартфон
Время на игры: 7 - 14 часов в неделю
Телосложение: Среднее
Цвет волос: Русый(ая)
Цвет 

In [ ]:
import numpy as np
k = 10
c = 0
good = 0
for key in gamers:
    print('-'*20)
    g = gamers[key]
    b = svd.transform([g.descr_encoding])
    try:
        svd_matrix = svd.transform([gamers[nick].descr_encoding for nick in gamers_keys if gamers[nick].gender in gamers[key].wishes['gender'] and gamers[nick].age >= gamers[key].wishes['age_low'] and gamers[nick].age <= gamers[key].wishes['age_high']])
    except:
        continue
    best = np.argsort(cosine_similarity(svd_matrix, b).reshape(-1))[-k:][::-1]
    for i in best:
        c += 1
        if (key, gamers_keys[i]) in targets:
            good += 1
        print(key, gamers_keys[i], (key, gamers_keys[i]) in targets)
#         print(gamers[gamers_keys[i]].description)
#         print('-'*20)

--------------------
goodman_dd NastjaIvanova False
goodman_dd google_user_4487 False
goodman_dd goodman_dd False
goodman_dd Asker False
goodman_dd AleksejGejmer False
goodman_dd LizMajers False
goodman_dd google_user_3429 False
goodman_dd Veteran False
goodman_dd AnnaVajt False
goodman_dd google_user_2386 False
--------------------
Literally Asker False
Literally Meowmeow False
Literally theFUM3 False
Literally Tallesa False
Literally EgorHabarov False
Literally Your_Sister False
Literally DenisFeofanov False
Literally google_user_3388 False
Literally Sh0ckeR44 False
Literally goodman_dd False
--------------------
Cogneter KanZaki False
Cogneter LizMajers False
Cogneter Your_Sister False
Cogneter google_user_3856 False
Cogneter Sh0ckeR44 False
Cogneter WatermelonSeal False
Cogneter gawric False
Cogneter Veteran False
Cogneter Rexolik False
Cogneter AleksejGejmer False
--------------------
KanZaki KanZaki False
KanZaki DanilKarpenko False
KanZaki google_user_2716 False
KanZaki Yuji Fal

Sapphire362 WatermelonSeal False
Sapphire362 google_user_3696 False
Sapphire362 AjdosGulev False
Sapphire362 Shax False
Sapphire362 google_user_1768 False
Sapphire362 _Devil_ False
--------------------
Kreyvus Karishashy False
Kreyvus RainbowFox False
Kreyvus gawric False
Kreyvus google_user_4487 False
Kreyvus LesyaTyan False
Kreyvus google_user_3856 False
Kreyvus KanZaki False
Kreyvus Meowmeow False
Kreyvus GenaVasiliev False
Kreyvus DanilKarpenko False
--------------------
AnnaVajt LizMajers False
AnnaVajt HSTT False
AnnaVajt Kaita False
AnnaVajt _Devil_ False
AnnaVajt AnnaVajt False
AnnaVajt google_user_4487 False
AnnaVajt NikitaDunaevskij False
AnnaVajt google_user_3429 False
AnnaVajt gawric False
AnnaVajt hewie False
--------------------
google_user_2386 google_user_2853 False
google_user_2386 DanilKarpenko False
google_user_2386 CHYHKA False
google_user_2386 WatermelonSeal False
google_user_2386 RainbowFox False
google_user_2386 GenaVasiliev False
google_user_2386 Rexolik False
g

google_user_2253 Meowmeow False
google_user_2253 GenaVasiliev False
google_user_2253 Cogneter False
google_user_2253 google_user_2853 False
google_user_2253 DanilKarpenko False
google_user_2253 google_user_4487 False
google_user_2253 WatermelonSeal False
google_user_2253 Karishashy False
google_user_2253 Literally False
google_user_2253 RainbowFox False
--------------------
bianconeri WatermelonSeal False
bianconeri google_user_2854 False
bianconeri LesyaTyan False
bianconeri goodman_dd False
bianconeri google_user_2853 False
bianconeri Sh0ckeR44 False
bianconeri Literally False
bianconeri KanZaki False
bianconeri Cogneter False
--------------------
ObiWan goodman_dd False
ObiWan Darkness False
ObiWan Rexolik False
ObiWan LizMajers False
ObiWan google_user_4487 False
ObiWan Asker False
ObiWan google_user_3319 False
ObiWan google_user_3429 False
ObiWan LesyaTyan False
ObiWan google_user_4365 False
--------------------
Anko google_user_2854 False
Anko Granix False
Anko LesyaTyan False
An

Yuji Yuji False
Yuji google_user_3583 False
Yuji GhostHelios False
Yuji Your_Sister False
Yuji DenisFeofanov False
Yuji Halcon False
Yuji Kley False
Yuji KsenijaLebedeva False
Yuji AjdosGulev False
Yuji RomanSaraev False
--------------------
rb201513 google_user_2854 False
rb201513 Karishashy False
rb201513 google_user_4365 False
rb201513 DanilKarpenko False
rb201513 LesyaTyan False
rb201513 Sh0ckeR44 False
rb201513 gawric False
rb201513 google_user_4487 False
rb201513 Cogneter False
rb201513 KanZaki False
--------------------
google_user_3583 Combobreaker False
google_user_3583 VladimirDubrovin False
google_user_3583 gawric False
google_user_3583 Darkness False
google_user_3583 NikitaGorit False
google_user_3583 _Devil_ False
google_user_3583 LizMajers False
google_user_3583 DenisJUrnero False
google_user_3583 Enclave False
google_user_3583 ValerijRepchenko False
--------------------
empty LizMajers False
empty google_user_3856 False
empty LesyaTyan False
empty Sh0ckeR44 False
empty A

RomanSaraev Meowmeow False
RomanSaraev gawric False
RomanSaraev Cogneter False
RomanSaraev goodman_dd False
--------------------
HanteRina DenisFeofanov False
HanteRina google_user_3319 False
HanteRina DanilKarpenko False
HanteRina google_user_3429 False
HanteRina Darkness False
HanteRina google_user_2854 False
HanteRina google_user_4365 False
HanteRina theFUM3 False
HanteRina AleksejGejmer False
HanteRina Sh0ckeR44 False
--------------------
google_user_4692 DanilKarpenko False
google_user_4692 google_user_4365 False
google_user_4692 Veteran False
google_user_4692 goodman_dd False
google_user_4692 google_user_3856 False
google_user_4692 WatermelonSeal False
google_user_4692 GenaVasiliev False
google_user_4692 Sh0ckeR44 False
google_user_4692 KanZaki False
google_user_4692 Meowmeow False
--------------------
DenisMalyshev google_user_2854 False
DenisMalyshev Cogneter False
DenisMalyshev LesyaTyan False
DenisMalyshev gawric False
DenisMalyshev DanilKarpenko False
DenisMalyshev google_us

In [ ]:
good

0

In [ ]:
gamers[key].age

20

In [ ]:
any(gamers[key].age == 20 for key in gamers if gamers[key].gender == 'Девушка')

True

In [ ]:
gamers[key].wishes['gender']

'Парень, Девушка'

In [ ]:
def parse_one_person(login):
    ignore_info = [
        'Дата активности',
        'Возраст',
        'Имя',
        'Я',
        'Кого ищу',
        'День рождения моего избранника',
        'Цель поиска'
    ]

    _, info = parse_login(login)
    age = int(info['Возраст'].split()[0])
    name = info['Имя']
    gender = info['Я']
    wishes = {
        'gender': info['Кого ищу'],
        'age_low': int(info['День рождения моего избранника'].split()[1]),
        'age_high': int(info['День рождения моего избранника'].split()[3]),
        'purpose': info['Цель поиска'].split(', ')
    }
    description = '\n'.join([f'{k}: {v}' for k, v in info.items() if k not in ignore_info])
    gamer = Gamer(name=name, login=login, gender=gender, age=age, description=description, wishes=wishes)
    return gamer

g = parse_one_person('JUnusFajzaliev')

In [ ]:
catboostmodel.predict(gamers['google_user_3616'].descr_encoding.tolist() + parse_one_person('JUnusFajzaliev').descr_encoding.tolist())

0.010896709934355809

In [ ]:
me = Gamer()
find_new_friends(n=10)

In [ ]:
import pandas as pd
import ast
# Чтение файла CSV
df = pd.read_csv('gamers (1).csv')

# Создание новых колонок для каждого параметра
df['About Me'] = df['description'].apply(lambda x: x.split('Обо мне: ')[1].split('\n')[0] if 'Обо мне: ' in x else '')
df['Interests'] = df['description'].apply(lambda x: x.split('Мои интересы: ')[1].split('\n')[0] if 'Мои интересы: ' in x else '')
df['Skills'] = df['description'].apply(lambda x: x.split('В чём я хорош: ')[1].split('\n')[0] if 'В чём я хорош: ' in x else '')
df['City'] = df['description'].apply(lambda x: x.split('Город: ')[1].split('\n')[0] if 'Город: ' in x else '')
df['Looking For'] = df['description'].apply(lambda x: x.split('О том, кого я хочу найти: ')[1].split('\n')[0] if 'О том, кого я хочу найти: ' in x else '')
df['Favorite Games'] = df['description'].apply(lambda x: x.split('Любимые игры: ')[1].split('\n')[0] if 'Любимые игры: ' in x else '')
df['Favorite Game Genres'] = df['description'].apply(lambda x: x.split('Любимые жанры игр: ')[1].split('\n')[0] if 'Любимые жанры игр: ' in x else '')
df['Platforms'] = df['description'].apply(lambda x: x.split('Платформы: ')[1].split('\n')[0] if 'Платформы: ' in x else '')
df['Time for Games'] = df['description'].apply(lambda x: x.split('Время на игры: ')[1].split('\n')[0] if 'Время на игры: ' in x else '')
df['Build'] = df['description'].apply(lambda x: x.split('Телосложение: ')[1].split('\n')[0] if 'Телосложение: ' in x else '')
df['Hair Color'] = df['description'].apply(lambda x: x.split('Цвет волос: ')[1].split('\n')[0] if 'Цвет волос: ' in x else '')
df['Eye Color'] = df['description'].apply(lambda x: x.split('Цвет глаз: ')[1].split('\n')[0] if 'Цвет глаз: ' in x else '')
df['Smoking'] = df['description'].apply(lambda x: x.split('Курение: ')[1].split('\n')[0] if 'Курение: ' in x else '')
df['Alcohol'] = df['description'].apply(lambda x: x.split('Употребление алкоголя: ')[1].split('\n')[0] if 'Употребление алкоголя: ' in x else '')
df['Games_now'] = df['description'].apply(lambda x: x.split('Во что я играю сейчас: ')[1].split('\n')[0] if 'Во что я играю сейчас: ' in x else '')
df['Steam'] = df['description'].apply(lambda x: x.split('Steam: ')[1].split('\n')[0] if 'Steam: ' in x else '')

# Extract information from the 'wishes' column
df['Wishes'] = df['wishes'].apply(lambda x: ast.literal_eval(x))
df['Wished Gender'] = df['Wishes'].apply(lambda x: x['gender'])
df['Age Low'] = df['Wishes'].apply(lambda x: x['age_low'])
df['Age High'] = df['Wishes'].apply(lambda x: x['age_high'])
df['Purpose'] = df['Wishes'].apply(lambda x: ', '.join(x['purpose']))

# Drop the original 'wishes' column
df.drop(columns=['wishes'], inplace=True)
df.drop(columns=['Wishes'], inplace=True)
df.drop(columns=['description'], inplace=True)
# Вывод результата

df.to_csv('fin_gamers.csv', index=False)



In [ ]:
def find_common_elements(str1, str2):
    common_elements = []
    for char in str1:
        if char in str2 and char not in common_elements:
            common_elements.append(char)
    return common_elements

In [ ]:
import pandas as pd
import ast
df = pd.read_csv('fin_gamers.csv')

df.drop(columns=['login','Looking For', 'Platforms', 'Build', 'Hair Color', 'Eye Color', 'Smoking', 'Alcohol', 'Steam', 'Skills'], inplace=True)

# Изменение колонки 'City'
df['City'] = df['City'].apply(lambda x: str(x).split(',')[0])

# Разделение колонки 'Favorite Games'
games = df['Favorite Games'].str.get_dummies(sep=', ')
df = pd.concat([df, games], axis=1)
df.drop(columns=['Favorite Games'], inplace=True)

# Разделение колонки 'Favorite Game Genres'
game_genres = df['Favorite Game Genres'].str.get_dummies(sep=', ')
df = pd.concat([df, game_genres], axis=1)
df.drop(columns=['Favorite Game Genres'], inplace=True)


game_genres = df['Games_now'].str.get_dummies(sep=', ')
df = pd.concat([df, game_genres], axis=1)
df.drop(columns=['Games_now'], inplace=True)

# Разделение колонки 'Wished Gender'
wished_gender = df['Wished Gender'].str.get_dummies(sep=', ')
df = pd.concat([df, wished_gender], axis=1)
df.drop(columns=['Wished Gender'], inplace=True)

# Разделение колонки 'Purpose'
purpose = df['Purpose'].str.get_dummies(sep=', ')
df = pd.concat([df, purpose], axis=1)
df.drop(columns=['Purpose'], inplace=True)

df.fillna(0, inplace=True)
df.replace('nan', 0, inplace=True)

df = df.T.drop_duplicates().T
df.to_csv('_gamers.csv', index=False)

df

# Save the new DataFrame to a CSV file


,name,gender,age,About Me,Interests,City,Time for Games,Age Low,Age High,5Street,...,Tower of Fantasy,Valheim,Warhammer: Vermintide 2,World of Warcraft,Девушка,Парень,Влюбиться,Подружиться,Поиграть,Пообщаться
0,Илья,Парень,37,Сижу дома и ничего не делаю. ),"Старые игры, в основном Денди, Сега МД, Сони П...",Выкса,Меньше 7 часов в неделю,18,71,0,...,0,0,0,0,1,0,0,0,0,1
1,Ronan Djohua (Вадим),Парень,27,Тихо и мирно предпринимаю попытки (преимуществ...,"История, волонтерство, география, военная техн...",0,Больше 28 часов в неделю,16,40,0,...,0,0,0,0,1,1,1,1,1,1
2,Krator1,Парень,26,"Обычный парень, чуть-чуть хиккан,большую часть...","Игры, аниме, музыка, сирики, стримы, да и так,...",0,0,16,81,0,...,0,0,0,0,1,1,0,1,1,1
3,Baka_Sobacka,Парень,22,Одинокий романтик. В комнате полной людей я ча...,"Меня всегда интересовало фэнтези и магия, обож...",0,14 - 28 часов в неделю,20,30,0,...,0,0,0,0,0,1,1,1,1,1
4,Electronik,Парень,31,0,0,0,14 - 28 часов в неделю,20,40,0,...,0,0,0,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Sh0ckeR44,Парень,37,"Геймер, ретро геймер, стример, блогер, поэт - ...","Огромная вселенная игровой индустрии, также ст...",Саратов,Больше 28 часов в неделю,17,81,0,...,0,1,0,0,1,1,1,1,1,1
124,KanZaki,Девушка,27,0,0,Тверь,0,20,35,0,...,0,0,0,0,1,1,0,1,1,1
125,Александр,Парень,33,"Создал этот сайт, пытаюсь его развивать. :)","Программирование, геймдизайн, технологии, наук...",Москва,7 - 14 часов в неделю,18,30,0,...,0,0,0,0,1,0,0,1,1,1
126,Literally,Девушка,26,0,0,Москва,Меньше 7 часов в неделю,20,35,0,...,0,0,0,0,0,1,0,1,1,1


In [ ]:
df = pd.read_csv('_gamers.csv')

# Преобразование столбца 'login' с помощью метода get_dummies()
df = pd.get_dummies(df, columns=['login'], prefix='', prefix_sep='')

df.to_csv('_gamers_.csv', index=False)
# Выводим результат
df

,name,gender,age,About Me,Interests,City,Age Low,Age High,Alan Wake,Apex Legends,...,google_user_433,google_user_4365,google_user_4487,google_user_4673,google_user_4692,hayaati77,hewie,mitsu,rb201513,theFUM3
0,Илья,0,37,Сижу дома и ничего не делаю. ),"Старые игры, в основном Денди, Сега МД, Сони П...",Выкса,18,71,0,0,...,False,False,False,False,False,False,False,False,False,False
1,Ronan Djohua (Вадим),0,27,Тихо и мирно предпринимаю попытки (преимуществ...,"История, волонтерство, география, военная техн...",0,16,40,0,0,...,False,False,False,False,False,False,False,False,False,False
2,Krator1,0,26,"Обычный парень, чуть-чуть хиккан,большую часть...","Игры, аниме, музыка, сирики, стримы, да и так,...",0,16,81,0,1,...,False,False,False,False,False,False,False,False,False,False
3,Baka_Sobacka,0,22,Одинокий романтик. В комнате полной людей я ча...,"Меня всегда интересовало фэнтези и магия, обож...",0,20,30,0,0,...,False,False,False,False,False,False,False,False,False,False
4,Electronik,0,31,0,0,0,20,40,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Sh0ckeR44,0,37,"Геймер, ретро геймер, стример, блогер, поэт - ...","Огромная вселенная игровой индустрии, также ст...",Саратов,17,81,0,0,...,False,False,False,False,False,False,False,False,False,False
124,KanZaki,1,27,0,0,Тверь,20,35,0,0,...,False,False,False,False,False,False,False,False,False,False
125,Александр,0,33,"Создал этот сайт, пытаюсь его развивать. :)","Программирование, геймдизайн, технологии, наук...",Москва,18,30,0,0,...,False,False,False,False,False,False,False,False,False,False
126,Literally,1,26,0,0,Москва,20,35,0,0,...,False,False,False,False,False,False,False,False,False,False
